In [13]:
from deepface import DeepFace
import pandas as pd
import numpy as np
from tqdm import tqdm
import dlib

ModuleNotFoundError: No module named 'dlib'

In [3]:
df = pd.read_csv('./fairface_label_val.csv')
df.head()

,file,age,gender,race,service_test
0,val/1.jpg,3-9,Male,East Asian,False
1,val/2.jpg,50-59,Female,East Asian,True
2,val/3.jpg,30-39,Male,White,True
3,val/4.jpg,20-29,Female,Latino_Hispanic,True
4,val/5.jpg,20-29,Male,Southeast Asian,False


In [4]:
pd.get_dummies(df.race).columns.values

array(['Black', 'East Asian', 'Indian', 'Latino_Hispanic',
       'Middle Eastern', 'Southeast Asian', 'White'], dtype=object)

In [5]:
age_dic = {'0-2':[0,2], '10-19':[10,19], '20-29':[20,29], '3-9':[3,9], '30-39':[30,39], '40-49':[40,49], '50-59':[50,59], '60-69':[60,69],
       'more than 70':[70,1000]}
gender_dic = {'Female':0, 'Male':1}
race_dic = {'Black':0, 'East Asian':1, 'Indian':2, 'Latino_Hispanic':3,
       'Middle Eastern':4, 'Southeast Asian':5, 'White':6}

In [6]:
all_results = np.zeros((len(gender_dic),len(race_dic)))
correct_results = np.zeros((len(gender_dic),len(race_dic)))
not_dect_results = np.zeros((len(gender_dic),len(race_dic)))

In [11]:
objs = DeepFace.analyze(img_path = df.iloc[0].file, 
            actions = ['age'], detector_backend = backends[2]
        )

ModuleNotFoundError: No module named 'dlib'

In [18]:
backends = [
  'opencv', 
  'ssd', 
  'dlib', 
  'mtcnn', 
  'retinaface', 
  'mediapipe'
]

for i in tqdm(range(len(df))):
    img = df.iloc[i].file
    true_age = df.iloc[i].age
    min_age, max_age = age_dic[true_age]
    gender = gender_dic[df.iloc[i].gender]
    race = race_dic[df.iloc[i].race]
    try:
        objs = DeepFace.analyze(img_path = df.iloc[i].file, 
            actions = ['age'], detector_backend = backends[4]
        )
        pred_age = objs[0]['age']
        if min_age < pred_age < max_age:
            correct_results[gender,race] += 1
        all_results[gender,race] += 1
    except:
        not_dect_results[gender,race] += 1


  0%|          | 0/10954 [00:00<?, ?it/s]

retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /home/jiazhengli/.deepface/weights/retinaface.h5
100%|██████████| 119M/119M [00:07<00:00, 15.2MB/s]
 12%|█▏        | 1306/10954 [01:10<07:38, 21.04it/s]

In [15]:
correct_results

array([[2., 1., 2., 1., 0., 1., 3.],
       [0., 1., 0., 0., 4., 0., 1.]])

In [16]:
all_results

array([[ 7.,  8.,  4.,  5.,  1.,  5., 10.],
       [ 4., 10.,  5., 10.,  6.,  8., 12.]])

In [17]:
not_dect_results

array([[1507., 1538., 1522., 1655.,  791., 1355., 1916.],
       [1594., 1544., 1501., 1576., 1620., 1462., 2232.]])

In [16]:
acc = correct_results / all_results

In [17]:
acc

array([[0.22680412, 0.26453488, 0.23692308, 0.19553073, 0.25153374,
        0.2260274 , 0.28448276],
       [0.18518519, 0.32093023, 0.26027397, 0.21176471, 0.26794258,
        0.27947598, 0.24414716]])

In [19]:
np.mean(acc, axis=0)

array([0.20599465, 0.29273256, 0.24859852, 0.20364772, 0.25973816,
       0.25275169, 0.26431496])

In [20]:
np.std(np.mean(acc, axis=0))

0.029640267755986088

In [27]:
np.log10((np.max(acc) / np.min(acc)))

0.2388043906446185